## VideoProcessing Tutorial

In this tutorial, the code is specifically designed for Neural Decoding Project. Main functionalities are:
1. Draw the field of view for mouse behaviral videos
2. Rotate the video by user-defined angles
3. Crop the video into actural mouse moving edge
4. Read and write video based on the video file name

Parameters and features for each functions are explained below, note that fieldOfView() has to collaborate with mouse location data to graph the sector in the video

### Calling package

In [8]:
import numpy as np 
import os
import glob
import pandas as pd
import h5py
from math import degrees, atan2
import cv2
from collections import deque

os.chdir("/home/donghan/DeepLabCut/data/") #Change to your own directory

### Read location data from H5 file

In [9]:
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[200:] 

### Functions
#### Head directions that used in drawing the field of view

In [3]:
def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    if __name__ == "__main__": 
        return (direction, degreeL)
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.


In [6]:
# Colors (B, G, R)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Draw the sector of field of view, parameters explained in fieldOfView()
def drawSector(image, radius, center, angle, startAngle, endAngle, color, thickness):
    # Ellipse parameters
    axes = (radius, radius)

    cv2.ellipse(image, center, axes, angle, startAngle, endAngle, color, thickness)
    points = cv2.ellipse2Poly(center, axes, angle, startAngle, endAngle, thickness)
    point1 = tuple(points[0]) # End's coordinates
    point2 = tuple(points[-1]) # Another end's coordinates
    cv2.line(image, center, point1, color)
    cv2.line(image, center, point2, color)
    return image

# Rotate image by certain degrees
def rotate(image, angle, center=None, scale=1): 
    #scale = 1: original size
    rows,cols,ch = image.shape
    if center == None:
        center = (cols / 2, rows / 2)
    M = cv2.getRotationMatrix2D(center, angle, scale) 
    #Matrix: Rotate with center by angles
    dst = cv2.warpAffine(image,M,(cols,rows)) 
    #After rotation
    return dst



def videoWriter(task, filename, outputName, corrX = None, corrY = None, dist = 320, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass


    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (dist, dist)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out


def videoEditer(filename, outputName, corrX, corrY, dist = 320, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + dist
    corrY1 = corrY + dist

    video = videoWriter("edited", filename, outputName, corrX, corrY, dist, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

def videoRotator(filename, outputName, rotationAngle, displayVideo = False):

    '''
    Task: Rotate video by user-designed degree (rotationAngle), by clockwise

    '''

    #cap: Read video
    #out: Write video
    video = videoWriter("rotated", filename, outputName, displayVideo)
    cap = video[0]
    out = video[1]    

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    #Read videos and rotate by certain degrees
    while(cap.isOpened()):
        #Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = rotate(img, rotationAngle) 
            #Flipped Vertically
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('rotated video',img2) 

            # Break the loop when the last frame has been rotated
            if count == frameCnt:
                print (filename, 'successfully ', "rotated!")
                break
            else:
                count += 1

            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()



def fieldOfView(data, filename, outputName, radius, rotationAngle, color, viewRange, thickness, displayVideo = False):
    '''    
    Task: draw sector for field of view

    All input parameters must be Int

    PARAMETERS:

    data: mouse movement that read from .h5 file, including head and tail. 

    angle: 0, rotation angle

    radius: radius of the sector

    color: sector line color

    viewRange: range that the mouse has vision on

    thickness: the thickness of the sector line

    '''

    #Obtain coordinates data for head and tail, as well as the head direction by degrees

    head = zip(round(data['x']).astype(int), round(data['y']).astype(int))
    tail = zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int))
    degrees = head_dir(data)[1]
    
    #cap: Read video
    #out: Write video
    video = videoWriter("field of view", filename, outputName, displayVideo)
    cap = video[0]
    out = video[1]

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    while(cap.isOpened()):
        try:
            for center, angle in zip(head, degrees):
                ret,img = cap.read()
            #draw the sector representing the field of view of mouse
                angle = int(round(angle))
                startAngle = angle - viewRange
                endAngle = angle + viewRange
                img2 = drawSector(img, radius, center, rotationAngle, startAngle, endAngle, color, thickness)
                out.write(img2)

            # Break the loop when the last frame has been rotated
            if count == frameCnt:
                print (filename, 'successfully ', "added field of view!!!")
                break
            else:
                count += 1
            if displayVideo == True:
                cv2.imshow('Draw field of view video',img2) 

            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()


    
def walkingPath(data, filename, outputName, length, color, displayVideo = False):
    '''
    Task: Graph the walking path of the mouse movement
    
    PARAMETERS:
    
    data: mouse location extracted from DeepLabCut
    
    filename: the video file that you would like to process
    
    outputName: filename of your output processed video
    
    length: length of the mouse trace
    
    color: mouse trace color
    
    '''

    head = list(zip(round(data['x']).astype(int), round(data['y']).astype(int)))
    tail = list(zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int)))

    #cap: Read video
    #out: Write video
    video = videoWriter("walking path", filename, outputName)
    cap = video[0]
    out = video[1]

    pts = deque(maxlen= length)
    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
            
    count = 1
    for i in range(0,len(tail)):
        ret,img = cap.read()
        cv2.circle(img, tail[i], 5, color, -1)
        # 5: radius of the circle, by default
        pts.appendleft(tail[i])

        for i in np.arange(1, len(pts)):
            thickness = int(np.sqrt(length / float(i + 1)) * 2.5)
            cv2.line(img, pts[i - 1], pts[i], color, thickness)
        out.write(img)

        if count == frameCnt:
            print(filename + " successfully added walking path")
            break
        else:
            count += 1
        
        if displayVideo == True:
            cv2.imshow("walking path", img)
        else:
            continue

        k=cv2.waitKey(30) & 0xff
        #once you inter Esc capturing will stop
        if k==27:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

In [10]:
from time import time
start = time()
walkingPath(data, "1034 SI_B, Aug 15, 13 7 49.mp4", "walkingPathTest.mp4",10, (198, 178, 158))
print("Runing time: %s" % str(time() - start))

1034 SI_B, Aug 15, 13 7 49.mp4 successfully added walking path
Runing time: 11.432162046432495


In [7]:
# General Procedure: field of view (location data needed, not listed here) -> rotate -> crop
# Change the working directory to the video data folder
def main():
    print("We have number of " + str(len(glob.glob("*.mp4"))) + " videos in total")
    i = 0
    for file in glob.glob("*.mp4"):
        if file.find('rotated') == -1 or file.find('edited') == -1:
            i += 1
            print("We are currently processing %s th video" % str(i))
            name = os.path.splitext(file)[0]
            filename = file
            outputName_rotated = name + ' rotated' + '.mp4'
            videoRotator(filename = filename, outputName = outputName_rotated, rotationAngle = -3.5)

            outputName_rotated = './rotated/' + outputName_rotated
            outputName_rotated_edited = name + ' rotated' + ' edited' + '.mp4'
            videoEditer(filename = outputName_rotated, outputName = outputName_rotated_edited, corrX = 35, corrY = 65, dist = 320, displayVideo = False)
        else:
            print("This video has been processed, skip")
            pass
if __name__ == "__main__":
    main()

We have number of 9 videos in total
We are currently processing 1 th video
We are currently processing 2 th video
We are currently processing 3 th video
We are currently processing 4 th video
We are currently processing 5 th video
We are currently processing 6 th video
We are currently processing 7 th video
We are currently processing 8 th video
We are currently processing 9 th video
